In [1]:
import requests
from bs4 import BeautifulSoup
import json
from pandas import DataFrame as df
import pandas as pd

cities = []
cities_href = []
stores= []
store_links = []
store_url = ""

states_href = []
page = requests.get("https://www.familydollar.com/locations/")
soup = BeautifulSoup(page.content,"html.parser")

# parse html to get all the state links
states_info = soup.find_all(class_ = 'itemlist')
for state in states_info:
    states_href.append(state.a["href"])
    

for state in states_href:
    state_page = requests.get(state)
    state_soup = BeautifulSoup(state_page.content,"html.parser")

#     parse to get store links in a city
    city_list_parent = state_soup.find_all(class_ = 'itemlist')
    for d in city_list_parent:
        cities.append(d.a.get_text())
        cities_href.append(d.a['href'])  
    
#     parse to get a detailed link
for store in cities_href:
    store_page = requests.get(store)
    store_soup = BeautifulSoup(store_page.text,"html.parser")


    store_info = store_soup.find_all(type="application/ld+json")    
    for i in store_info:
        storecont = i.contents[0]
        storejson = json.loads(storecont)

    try:
        store_url = storejson['url']    
        store_links.append(store_url)            
    except:
        pass

    #     parse to get address and geo code of each store
    store_geopage = requests.get(store_url)
    store_geosoup = BeautifulSoup(store_geopage.text,"html.parser")

    store_geoinfo = store_geosoup.find_all(type="application/ld+json") 
    for geo_i in store_geoinfo:
        store_geocont = geo_i.contents[0]
        store_geojson = json.loads(store_geocont)
        try:
            store_geoaddr = store_geojson['address']            
            store_geoaddr.update(store_geojson['geo'])
            store_geoaddr["link"] = store_geojson['url']
            stores.append(store_geoaddr)
        except:
            pass   

stores_df = df.from_records(stores)
# drop unnecessary info from dataframe
stores_df.drop(['@type', 'addressCountry'], axis = 1, inplace = True)
stores_df['Store'] = "Family Dollar"
stores_df.to_csv("FamilyDollar-AllStores.csv")